# Approve DTA Version (Create DTA Major)

Promotes the current draft from Silver to DTA Major in Gold library.

**Operation:** `APPROVE_DTA` / `CREATE_DTA_MAJOR`

**Modes:**
- `HISTORICAL`: Promotes draft to DTA Major (auto-approval for historical imports)
- `UI`: Promotes draft to DTA Major (triggered by workflow approval)

**What it does:**
1. Gets DTA ID(s) from widget or task values
2. Reads approved draft records from Silver
3. Aggregates and creates DTA Major version in Gold library (per DTA, 85 records each)
4. Updates Silver records: `version_status=APPROVED`
5. Updates registry: draft → SUPERSEDED, DTA_MAJOR → ACTIVE
6. Updates DTA entity: `latest_major_version`, `current_version_tag`

**Version format:** `{base}-{dta_number}-v1.0` (e.g., `1.0-DTA001-v1.0`)

**Parameters:**
- `dta_id` - Single DTA identifier (for UI/API call)
- `source` - HISTORICAL or UI
- `library_type` - Library type (e.g., transfer_variables)


In [ ]:
# Cell 1: Imports
import json
from datetime import datetime
from pyspark.sql import functions as F
from clinical_data_standards_framework.versioning import get_version_registry_schema

print("="*80)
print("APPROVE DTA VERSION (Create DTA Major)")
print("="*80)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
versioning_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="versioning"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
silver_schema = globals_dict['silver_schema']
gold_schema = globals_dict['gold_schema']

# Versioning configuration from YAML
library_tables = versioning_dict.get('library_tables', [])
registry_table_name = versioning_dict.get('registry_table', 'md_version_registry')

print(f"Catalog: {catalog}")
print(f"Silver Schema: {silver_schema}")
print(f"Gold Schema: {gold_schema}")
print(f"Created by: {created_by_principal}")


In [ ]:
# Cell 3: Get Widget Parameters and DTA IDs
dbutils.widgets.text("dta_id", "", "DTA ID (single, for UI)")
dbutils.widgets.text("source", "UI", "Source (HISTORICAL or UI)")
dbutils.widgets.text("library_type", "", "Library Type")

dta_id_param = dbutils.widgets.get("dta_id")
source = dbutils.widgets.get("source")
library_type_param = dbutils.widgets.get("library_type")

# Determine library type
if library_type_param:
    library_type = library_type_param
elif library_tables:
    library_type = library_tables[0].get('library_type', 'transfer_variables')
else:
    library_type = 'transfer_variables'

# Get DTA IDs: Priority order:
# 1. Widget parameter (for UI/API calls via version manager)
# 2. Task values from save_draft (for HISTORICAL flow in job_cdm_dta_create)
# 3. Task values from create_dta_instance (fallback for HISTORICAL flow)
dta_ids = []

if dta_id_param:
    # Called via version manager or API with explicit dta_id
    dta_ids = [dta_id_param]
    print(f"Mode: Single DTA from widget parameter")
else:
    # Try to get from task values (HISTORICAL flow)
    try:
        dta_ids = json.loads(dbutils.jobs.taskValues.get(taskKey="save_draft", key="dta_ids"))
        print(f"Mode: Multiple DTAs from save_draft task values")
    except Exception:
        try:
            dta_ids = json.loads(dbutils.jobs.taskValues.get(taskKey="create_dta_instance", key="created_dta_ids"))
            print(f"Mode: Multiple DTAs from create_dta_instance task values")
        except Exception:
            # No task values - this is expected when called via version manager
            pass

# Validate we have at least one DTA ID
if not dta_ids:
    raise ValueError(
        "No dta_id provided. When calling APPROVE_DTA via job_cdm_version_manager, "
        "you must provide the dta_id parameter."
    )

print(f"\nSource: {source}")
print(f"Library Type: {library_type}")
print(f"DTAs to process: {len(dta_ids)}")


In [ ]:
# Cell 4: Configure Table Names
silver_table = f"{catalog}.{silver_schema}.md_transfer_variable_field_normalized"
dta_table = f"{catalog}.{gold_schema}.dta"
registry_table = f"{catalog}.{gold_schema}.{registry_table_name}"

# Build library table from config
library_table_map = {}
for lib_config in library_tables:
    lib_type = lib_config.get('library_type')
    lib_name = lib_config.get('name')
    lib_schema = lib_config.get('schema', gold_schema)
    if lib_type and lib_name:
        library_table_map[lib_type] = f"{catalog}.{lib_schema}.{lib_name}"

library_table = library_table_map.get(library_type, f"{catalog}.{gold_schema}.md_transfer_variables_library")

print(f"\nSilver Table: {silver_table}")
print(f"Library Table (Gold): {library_table}")
print(f"DTA Table: {dta_table}")
print(f"Registry Table: {registry_table}")


In [ ]:
# Cell 5: Load DTA Metadata
now = datetime.now()
base_library_version = "1.0"

df_dta = spark.table(dta_table).filter(F.col("dta_id").isin(dta_ids))
dta_metadata = {row["dta_id"]: row.asDict() for row in df_dta.collect()}
print(f"\nLoaded metadata for {len(dta_metadata)} DTAs")

# Get draft version tags from save_draft or DTA metadata
try:
    draft_versions = json.loads(dbutils.jobs.taskValues.get(taskKey="save_draft", key="draft_versions"))
except:
    draft_versions = {dta_id: dta_metadata.get(dta_id, {}).get("current_draft_version") for dta_id in dta_ids}


In [ ]:
# Cell 6: Create DTA Major Versions in Gold Library
print(f"\n{'='*80}")
print("Creating DTA Major versions in Gold library...")
print(f"{'='*80}")

major_versions = {}
registry_records = []

for dta_id in dta_ids:
    meta = dta_metadata.get(dta_id, {})
    dta_number = meta.get("dta_number", "DTA000")
    draft_version = draft_versions.get(dta_id, f"{base_library_version}-{dta_number}-draft1")
    
    # Create major version tag
    major_version_tag = f"{base_library_version}-{dta_number}-v1.0"
    major_versions[dta_id] = major_version_tag
    
    print(f"\nProcessing DTA: {dta_number}")
    print(f"  Draft: {draft_version} → Major: {major_version_tag}")
    
    # Read draft records from Silver
    df_draft = spark.table(silver_table).filter(
        (F.col("dta_id") == dta_id) & 
        (F.col("version_status") == "DRAFT")
    )
    draft_count = df_draft.count()
    print(f"  Silver draft records: {draft_count}")
    
    # Update Silver: mark as APPROVED
    spark.sql(f"""
        UPDATE {silver_table}
        SET version_status = 'APPROVED',
            version_tag = '{major_version_tag}',
            is_current_draft = false
        WHERE dta_id = '{dta_id}' AND version_status = 'DRAFT'
    """)
    print(f"  Updated silver records to APPROVED")


In [ ]:
# Cell 7: Copy Records to Gold Library
print(f"\n{'='*80}")
print("Copying DTA Major records to Gold library...")
print(f"{'='*80}")

# Define gold library schema columns to select from silver
gold_columns = [
    "definition_hash", "transfer_variable_name", "transfer_variable_label", "format",
    "anticipated_max_length", "transfer_file_key", "populate_for_all_records", 
    "codelist_values", "variable_description", "example_values", "row_status",
    "dta_id", "version_tag"
]

for dta_id in dta_ids:
    meta = dta_metadata.get(dta_id, {})
    major_version_tag = major_versions[dta_id]
    
    # Read approved records
    df_approved = spark.table(silver_table).filter(
        (F.col("dta_id") == dta_id) & (F.col("version_status") == "APPROVED")
    )
    
    # Select and add gold-specific columns
    df_gold = df_approved.select(
        *[F.col(c) for c in gold_columns if c in df_approved.columns]
    ).withColumn("library_version", F.lit(major_version_tag)
    ).withColumn("is_major_version", F.lit(True)
    ).withColumn("is_dta_major", F.lit(True)
    ).withColumn("parent_version", F.lit(base_library_version)
    ).withColumn("effective_start_ts", F.lit(now)
    ).withColumn("effective_end_ts", F.lit(None).cast("timestamp")
    ).withColumn("is_current", F.lit(True))
    
    # Append to gold library
    df_gold.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable(library_table)
    gold_count = df_gold.count()
    print(f"  {meta.get('dta_number')}: Added {gold_count} records to gold library")


In [ ]:
# Cell 8: Register Versions and Update DTA Entities
print(f"\n{'='*80}")
print("Registering versions and updating DTA entities...")
print(f"{'='*80}")

for dta_id, major_version_tag in major_versions.items():
    meta = dta_metadata.get(dta_id, {})
    draft_version = draft_versions.get(dta_id)
    
    # Get record count
    record_count = spark.table(library_table).filter(
        F.col("library_version") == major_version_tag
    ).count()
    
    # Update draft registry entry to SUPERSEDED
    if draft_version:
        spark.sql(f"""
            UPDATE {registry_table}
            SET status = 'SUPERSEDED', last_updated_ts = current_timestamp()
            WHERE version_tag = '{draft_version}' AND dta_id = '{dta_id}'
        """)
    
    # Create DTA_MAJOR registry entry
    registry_record = {
        "version_tag": major_version_tag,
        "library_type": library_type,
        "version_type": "DTA_MAJOR",
        "is_latest_major": False,
        "dta_id": dta_id,
        "dta_trial_id": meta.get("trial_id"),
        "dta_stream_type": meta.get("data_stream_type"),
        "dta_provider_name": meta.get("data_provider_name"),
        "parent_version": base_library_version,
        "record_count": record_count,
        "status": "ACTIVE",
        "notes": f"DTA Major created from {source}",
        "created_by_principal": created_by_principal,
        "created_ts": now,
        "last_updated_by_principal": created_by_principal,
        "last_updated_ts": now,
        "databricks_job_id": databricks_job_id,
        "databricks_job_name": databricks_job_name,
        "databricks_run_id": databricks_run_id
    }
    registry_df = spark.createDataFrame([registry_record], schema=get_version_registry_schema())
    registry_df.write.format("delta").mode("append").saveAsTable(registry_table)
    
    # Update DTA entity
    spark.sql(f"""
        UPDATE {dta_table}
        SET current_version_tag = '{major_version_tag}',
            latest_major_version = '{major_version_tag}',
            workflow_state = 'APPROVED',
            last_updated_ts = current_timestamp()
        WHERE dta_id = '{dta_id}'
    """)
    print(f"  {meta.get('dta_number')}: Registered {major_version_tag}")


In [ ]:
# Cell 9: Set Task Values and Summary
dbutils.jobs.taskValues.set(key="major_versions", value=json.dumps(major_versions))
dbutils.jobs.taskValues.set(key="dta_ids", value=json.dumps(dta_ids))
dbutils.jobs.taskValues.set(key="action", value="APPROVE_DTA")

print(f"\n{'='*80}")
print("SUMMARY")
print(f"{'='*80}")
print(f"Source: {source}")
print(f"DTAs approved: {len(major_versions)}")
for dta_id, version in major_versions.items():
    meta = dta_metadata.get(dta_id, {})
    print(f"  {meta.get('dta_number')}: {version}")
print(f"\n✅ DTA Major versions created successfully!")
